# **Qué es un índice internamente**

**En SQLite es una estructura tipo B-Tree que:**

- Mantiene los valores ordenados

- Permite búsquedas logarítmicas

- Evita recorrer toda la tabla

# **Consulta sin indice**

En una consulta sin indice 

**SQLite:**

1. Recorre toda la tabla 

2. Evalúa fila por fila:

- condicion 1

- condicion 2

- etc

3. Cuenta coincidencias

Esto se llama full table scan.

**Por qué es costoso?**

- Si hay 10 filas → casi instantáneo

- Si hay 100.000 filas → lento

- Si hay millones → crítico

# **Consulta con Indice**

- El índice no es genérico, está pensado para una consulta concreta.

Qué es un índice internamente?

**En SQLite es una estructura tipo B-Tree que:**

- Mantiene los valores ordenados

- Permite búsquedas logarítmicas

- Evita recorrer toda la tabla


            IF NOT EXISTS

**Por qué se usa?**

- Evita error si el índice ya existe.

Esto es clave en:

- Scripts reutilizables

- Ejecuciones múltiples

- Entornos productivos


            conn.commit()

**Qué hace?**

Confirma la creación del índice.

**Por qué es obligatorio**

1. Los índices también son persistencia

2. Sin commit:

- El índice no se guarda

- No se usa en la consulta siguiente

# **Qué pasa ahora con índice**

SQLite:

1. Usa (directamente el indice creado)

2. Salta directamente a "condicion 1"

3. Dentro de ese subconjunto, evalúa "condicion 2"

4. Cuenta sin recorrer la tabla completa

**Resultado**

- Menos I/O

- Menos CPU

- Mucho más rápido

# **El uso de indices deberia ser para optimizar el timepo**

Tablas pequeñas → mejora leve

Tablas grandes → mejora brutal (10x, 100x)

**1. Los índices no son mágicos**

Sirven solo para consultas frecuentes

Tienen costo de escritura

**2. Orden del índice importa**

**3. Índices = trade-off**

✔ Lecturas más rápidas

x Inserts/updates un poco más lentos

x Más espacio en disco

# **Cuándo NO usar índices?**

Esta es una pregunta de nivel profesional. Saber cuándo NO usar índices es tan importante como saber crearlos.

**Idea central**

- Un índice acelera lecturas, pero penaliza escrituras y consume recursos.

- Si no hay beneficio claro, el índice es un costo innecesario.

# **1. NO usar índices en tablas pequeñas**

Por qué

**Si la tabla tiene pocas filas, el motor:**

- Lee toda la tabla más rápido

- Evita el costo de usar el índice

Con 20 filas:

Full scan ≈ instantáneo

Índice ≈ overhead innecesario

**Regla práctica**

Menos de algunas miles de filas → generalmente no vale la pena.

# **2. NO indexar columnas con pocos valores distintos**

Esto se llama **baja selectividad.**

**Ejemplos malos para índices**

- estado → activo / inactivo

- sexo → M / F

- disponible → true / false

**Por qué**

El índice no filtra casi nada:

- Devuelve demasiadas filas

- El motor termina leyendo casi toda la tabla igual

Si el 95% es true, el índice no ayuda

# **3. NO usar índices en columnas que cambian constantemente**

**Por qué**

Cada:

- INSERT

- UPDATE

- DELETE

Debe actualizar:

- La tabla

- Cada índice asociado

Esto:

- Aumenta el tiempo de escritura

- Genera fragmentación

- Consume CPU y disco

**Datos muy volátiles → mal candidatos a índices.**

# **4.NO indexar columnas que no se usan en WHERE / JOIN / ORDER BY**

**Regla de oro**

Si una columna no aparece en consultas, **no merece un índice.**

El índice es espacio desperdiciado.

# **5. NO indexar columnas con valores únicos “naturales” largos**

Ejemplo

- Emails

- URLs

- Descripciones largas

- Textos

**Por qué**

- Índices grandes

- Comparaciones lentas

- Mayor uso de memoria

Por eso se usan **IDs numéricos** como claves.

# **6. NO crear índices “por las dudas”**

Este es un error muy común.

**Mala práctica**

“Capaz que algún día esta consulta sea lenta…”

**Resultado:**

- Muchos índices

- Escrituras lentas

- Mantenimiento costoso

- Planificador confundido

# **7. NO usar índices en tablas intermedias pequeñas (a veces)**

Si:

- Tiene pocas filas

- Se usa poco

Un índice puede ser innecesario.

**Aunque en tablas N–N grandes sí conviene.**

# **8. NO indexar columnas que siempre se leen completas**

Si:

- Siempre se lee todo

- Nunca se filtra

El índice no aporta nada.


# **9. NO usar índices en sistemas con muchas escrituras y pocas lecturas**

Ejemplo

- Logs en tiempo real

- Métricas

- Telemetría

**Ahí se prioriza:**

- Velocidad de escritura

- Insert masivo

- Índices mínimos

# **10. NO indexar sin medir antes y después**

**Error clásico**

Crear índice y asumir mejora.

**Forma correcta**

1. Medir sin índice

2. Crear índice

3. Medir con índice

4. Comparar

Si no mejora → eliminarlo.

# **Tabla resumen (rápida)**

|Situación	|Índice|
|:-:|:-:|
|Tabla pequeña	|❌|
|Pocos valores distintos	|❌|
|Columna muy volátil	|❌|
|No usada en consultas	|❌|
|Texto largo	|❌|
|Alta escritura / baja lectura	|❌|
|Consulta frecuente y filtrada	|✅|
|JOIN frecuente	|✅|
|WHERE selectivo	|✅|